# Peer Graded Assignment: Week 3:Part-1


In [60]:
#!conda install -c conda-forge geopy --yes

In [10]:
import pandas as pd
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
from pandas.io.json import json_normalize

import numpy as np

from geopy.geocoders import Nominatim

from bs4 import BeautifulSoup

from urllib.request import urlopen
import requests

import folium

import matplotlib.cm as cm

import matplotlib.colors as colors

import json

from sklearn.cluster import KMeans


print("Imported required libraries successfully!")




Imported required libraries successfully!


## 2. Scrape the data from
## https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [16]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page=urlopen(url).read().decode('utf-8')
soup=BeautifulSoup(page,'html.parser')

postal_code_table_html=soup.body.table.tbody

## 3. extract data and convert it to pandas dataframe

In [19]:
def extract_cell(elem):
    row=[]
    
    for cell in elem.find_all('td'):
        if (cell.a):
            if(cell.a.text):
                row.append(cell.a.text)
                continue
        row.append(cell.string.strip())
            
    return row
    

In [21]:
def extract_row():
    data=[]
    
    for tr in postal_code_table_html.find_all('tr'):
        row=extract_cell(tr)
        if len(row) !=3:
            continue
        data.append(row)
        
    return data
    

### a. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [22]:
data=extract_row()
columns=["Postcode","Borough","Neighbourhood"]
df=pd.DataFrame(data,columns=columns)

In [23]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [26]:
df.shape

(287, 3)

### b. Ignore cells with a borough that is Not assigned.

In [28]:
df1=df[df.Borough != 'Not assigned']
df1=df1.sort_values(by=['Postcode','Borough'])
df1.reset_index(inplace=True)
df1.head()

,index,Postcode,Borough,Neighbourhood
0,10,M1B,Scarborough,Rouge
1,11,M1B,Scarborough,Malvern
2,26,M1C,Scarborough,Highland Creek
3,27,M1C,Scarborough,Rouge Hill
4,28,M1C,Scarborough,Port Union


### d. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [56]:
df1.loc[df1['Neighbourhood'] =='Not assigned' , 'Neighbourhood'] = df1['Borough']
df1.head(10)

,index,Postcode,Borough,Neighbourhood
0,10,M1B,Scarborough,Scarborough
1,11,M1B,Scarborough,Scarborough
2,26,M1C,Scarborough,Highland Creek
3,27,M1C,Scarborough,Rouge Hill
4,28,M1C,Scarborough,Port Union
5,41,M1E,Scarborough,Guildwood
6,42,M1E,Scarborough,Morningside
7,43,M1E,Scarborough,West Hill
8,52,M1G,Scarborough,Scarborough
9,61,M1H,Scarborough,Cedarbrae


### c.Rows will be same postalcode will combined into one row with the neighborhoods separated with a comma

In [57]:
result = df1.groupby(['Postcode','Borough'], sort=False).agg( ', '.join)

df2=result.reset_index()
df2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Scarborough, Scarborough"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Scarborough
4,M1H,Scarborough,Cedarbrae


In [59]:
df2.shape

(103, 3)